In [50]:
import os, os.path
import numpy as np
import pandas as pd
#from model_attributes import *
import model_attributes as ma
from attribute_table import AttributeTable
import model_afolu as mafl
import model_ippu as mi
import model_circular_economy as mc
import model_electricity as ml
import model_energy as me
import model_socioeconomic as se
from model_socioeconomic import Socioeconomic
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
from typing import Union
import inspect
import ingestion as ing
import logging
from sisepuede_file_structure import *
import auxiliary_definitions_transformations as adt

importlib.reload(ma)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(mafl)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)
importlib.reload(adt)


import sisepuede as ssp
def _setup_logger(namespace: str, fn_out: Union[str, None] = None) -> None:
    global logger
    
    format_str = "%(asctime)s - %(levelname)s - %(message)s"
    # configure
    if fn_out is not None:
        logging.basicConfig(
            filename = fn_out,
            filemode = "w",
            format = format_str,
            level = logging.DEBUG
        )
    else:
        logging.basicConfig(
            format = format_str,
            level = logging.DEBUG
        )
        
    logger = logging.getLogger(namespace)
    # create console handler and set level to debug
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter
    formatter = logging.Formatter(format_str)
    # add formatter to ch
    ch.setFormatter(formatter)
    # add ch to logger
    logger.addHandler(ch)

    return logger

_setup_logger(__name__, os.path.join(os.getcwd(), "log_temp_transform.log"))

<Logger __main__ (DEBUG)>

In [51]:
##########################
#   LOAD INPUT TABLES    #
##########################

warnings.filterwarnings("ignore")
importlib.reload(ml)
importlib.reload(me)

##  setup location of calibrated files by sector

df_fake_data = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
dir_calibs = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo"

# setup output templates 
dir_template_inputs = os.path.join(dir_calibs, "template_inputs")
os.makedirs(dir_template_inputs, exist_ok = True) if not os.path.exists(dir_template_inputs) else None
dict_template_file_paths = {}

dict_calibration_file_paths = {
    "af": os.path.join(dir_calibs, "af", "data_complete_future_2022_09_30_test_updated.csv"),
    "ce": os.path.join(dir_calibs, "ce", "data_complete_future_2022_12_13_test.csv"),
    "en": os.path.join(dir_calibs, "en", "data_complete_future_2023_02_24.csv"),#"data_complete_future_2022_12_09_test.csv"),
    "ip": os.path.join(dir_calibs, "ip", "data_ippu_2023_01_16_fixed_cleaned.csv"),
    # use most recent for socioeconomic
    "se": os.path.join(dir_calibs, "en", "data_complete_future_2023_02_24.csv")
}

"""
dict_calibration_file_paths = {
    "af": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv"),
    "ce": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv"),
    "en": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv"),#"data_complete_future_2022_12_09_test.csv"),
    "ip": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv"),
    # use most recent for socioeconomic
    "se": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv")
}
""";


# initialize 
all_regions = None
dict_isos = {}
dict_calibration_tables = {}
dict_replace_iso = sa.model_attributes.dict_attributes.get("region").field_maps.get("iso_alpha_3_to_region")
dict_sets = {}
field_country = "country"
field_iso = "iso_code3"
field_region = "nation"
field_year = "year"
fields_drop = [field_iso, field_region, field_year] # only apply later
field_time_period = sa.model_attributes.dim_time_period

# get some attributes
attr_region = sa.model_attributes.dict_attributes.get("region")
attr_sector = sa.model_attributes.dict_attributes.get("abbreviation_sector")
attr_strat = sa.model_attributes.dict_attributes.get(f"dim_{sa.model_attributes.dim_strategy_id}")
attr_time_period = sa.model_attributes.dict_attributes.get(f"dim_{sa.model_attributes.dim_time_period}")

# dictionary and attribute derivaties
dict_strat_to_strat_id = attr_strat.field_maps.get("strategy_to_strategy_id")
dict_repl_time_period = attr_time_period.field_maps.get(f"year_to_{sa.model_attributes.dim_time_period}")
strat_base = int(attr_strat.table[attr_strat.table["baseline_strategy_id"] == 1][attr_strat.key])
# map each country to ISO code 3 and each code to 
dict_country_to_iso = dict((k, v.upper()) for k, v in attr_region.field_maps.get(f"{attr_region.key}_to_iso_alpha_3").items())
dict_iso_to_country = sf.reverse_dict(dict_country_to_iso)
dict_country_to_cf_region = attr_region.field_maps.get(f"{attr_region.key}_to_hourly_capacity_factor_region")
dict_iso_to_cf_region = dict((dict_country_to_iso.get(k), v) for k, v in dict_country_to_cf_region.items())
all_iso = list(dict_iso_to_country.keys())

# some time period oriented tools
dict_year_to_time_period = attr_time_period.field_maps.get(f"year_to_{attr_time_period.key}")
def year_to_tp(
    year: int
) -> int:

    m = min((dict_year_to_time_period.keys()))
    m_v = dict_year_to_time_period.get(m)

    diff = m - m_v

    return dict_year_to_time_period.get(year, year - diff)

# initialize relevant models
model_afolu = mafl.AFOLU(sa.model_attributes)
model_energy = me.NonElectricEnergy(sa.model_attributes)
model_elec = ml.ElectricEnergy(
    sa.model_attributes, 
    sa.dir_jl,
    sa.dir_ref_nemo,
    initialize_julia = False
)



#
#    SOME FUNCTIONS    #
#

# function to verify time periods
def check_time_periods(
    df_in: pd.DataFrame,
    attribute_time_period: AttributeTable,
    fields_group_by: List[str]
) -> pd.DataFrame:
    """
    Check a data frame to ensure that a record exists for each time period
        for the grouping specified in fields_group_by
    """
    
    # check fields
    fields_group_by = [x for x in fields_group_by if x in df_in.columns]
    if len(fields_group_by) == 0:
        return None
    
    field_time_period = attribute_time_period.key
    
    dfs = df_in.groupby(fields_group_by)
    df_out = []
    for df in dfs:
        i, df = df
        
        tps = sorted(list(df[field_time_period]))
        tps_set = sorted(list(set(tps)))
        if (tps == tps_set) & (set(tps) == set(attribute_time_period.key_values)):
            df_out.append(df) 
    
    return pd.concat(df_out, axis = 0).reset_index(drop = True)



def scale_vars(
    df_in: pd.DataFrame,
    dict_scale: Dict[str, float],
    model_attributes: ModelAttributes
) -> pd.DataFrame:
    """
    Scale model variables k in a data frame according to 
        dict_scale = {k: scalar}
    """
    
    df_out = df_in.copy()
    for modvar in dict_scale.keys():
        subsec = model_attributes.get_variable_subsector(modvar)
        fields = model_attributes.build_varlist(
            subsec,
            modvar
        ) if (subsec is not None) else None
        
        if fields is not None:
            df_out[fields] *= dict_scale.get(modvar)
        
    return df_out



########################
###                  ###
###    BEGIN TEMP    ###
###                  ###
########################

###
###    TEMPORARY: OVERWRITE CAPACITIES WITH EARLIER DATA
###

df_residual_capacity_overwrite = pd.read_csv(sa.fp_csv_nemomod_residual_capacity_inputs)
df_residual_capacity_overwrite = df_residual_capacity_overwrite[
    df_residual_capacity_overwrite["year"].isin(dict_repl_time_period.keys())
]

dict_replace_iso_rev = sf.reverse_dict(dict_replace_iso)
dict_replace_iso_rev = dict((k, v.upper()) for k, v in dict_replace_iso_rev.items())
df_residual_capacity_overwrite["year"].replace(dict_repl_time_period, inplace = True)
df_residual_capacity_overwrite["country"].replace(dict_replace_iso_rev, inplace = True)

df_residual_capacity_overwrite.rename(columns = {
    "year": sa.model_attributes.dim_time_period,
    "country": field_iso
}, inplace = True)


###
###  TEMPORARY: OVERWRITE AFOLU CLIMATE DATA AND SOC
###

# KCC
df_kcc = pd.read_csv(sa.fp_csv_climate_fields_by_country_simple)
df_kcc[field_iso] = df_kcc[field_country].replace(dict_country_to_iso)
df_kcc.drop([field_country], axis = 1, inplace = True)

# SOC
df_soc = pd.read_csv(sa.fp_csv_soc_fields_by_country_simple)
df_soc[field_iso] = df_soc[field_country].replace(dict_country_to_iso)
df_soc.drop([field_country], axis = 1, inplace = True)

df_climate = pd.merge(df_kcc, df_soc, how = "inner")


###
###  TEMPORARY: OVERWRITE SOME MORE ENERGY DATA
###

fp_data_overwrite = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo/en/data_complete_future_2023_04_02.csv"
df_overwrite = pd.read_csv(fp_data_overwrite)
fields_drop = [x for x in df_overwrite.columns if ("gas_furnace" in x) or ("gas_petroleum_liquid" in x)]
fields_drop += [x for x in df_overwrite.columns if ("residual_capacity" in x)]
fields_drop_ow = [x for x in df_overwrite.columns if ("variable_cost" in x) and ("per_gw" in x)]

df_overwrite[sa.model_attributes.dim_time_period] = df_overwrite["Year"].apply(year_to_tp)
df_overwrite.drop(fields_drop, axis = 1, inplace = True) if (len(fields_drop) > 0) else None

# merge/interpolate to deal with any missing years
df_all_rows = sf.explode_merge(
    attr_time_period.table[[attr_time_period.key]].drop_duplicates(),
    df_overwrite[[field_iso]].drop_duplicates()
)
df_overwrite = pd.merge(
    df_all_rows, 
    df_overwrite, 
    how = "left"
).interpolate().drop(["Year", "Nation"], axis = 1).sort_values(by = [field_iso, attr_time_period.key]).reset_index(drop = True)




###
###    TEMPORARY: OVERWRITE INITIAL PRODUCTION
###

df_prodvalues = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/ippu/ProductionValues_2023_02_17.csv")
field_tp = sa.model_attributes.dim_time_period
df_tp = sa.model_attributes.dict_attributes.get(f"dim_{field_tp}").table[[field_tp]]
df_prodvalues = sf.explode_merge(df_prodvalues, df_tp) if (field_tp not in df_prodvalues.columns) else df_prodvalues
df_prodvalues["nation"].replace(dict_replace_iso_rev, inplace = True)
df_prodvalues.rename(
    columns = {
        "nation": field_iso
    }, 
    inplace = True
)


###
###    TEMPORARY: OVERWRITE SELECT EFFICIENCY FACTORS REPRESENTING IMPROVEMENTS IN ELECTRIFYING HEAT ENERGY TO 
###

fields_overwrite = sa.model_attributes.build_varlist(
    sa.model_attributes.subsec_name_enfu,
    "Average Industrial Energy Fuel Efficiency Factor",
    restrict_to_category_values = ["fuel_electricity"]
) + sa.model_attributes.build_varlist(
    sa.model_attributes.subsec_name_scoe,
    "SCOE Efficiency Factor for Heat Energy from Electricity"
)
df_effic_factors = df_fake_data[[field_time_period] + fields_overwrite]



###
###    TEMPORARY: CAP MAX TECH CAPACITY FOR HYDROPOWER
###

df_hydro_cap_synthetic = pd.read_csv(sa.fp_csv_nemomod_hydropower_max_tech_capacity)



###
###    TEMPORARY: READ IN MinShareProduction BASELINE VALUES
###


df_min_share_production = pd.read_csv(
    sa.fp_csv_nemomod_minimum_share_of_production_baselines
).drop(["year"], axis = 1)

if False:
    # temporary modification - try cutting the MSP in hal for the first 6 years (while capping new investments)
    df_min_share_production.loc[
        [x < 6 for x in list(df_min_share_production[sa.model_attributes.dim_time_period])], 
        [x for x in df_min_share_production.columns if "nemomod" in x]
    ] = df_min_share_production.loc[
        [x < 6 for x in list(df_min_share_production[sa.model_attributes.dim_time_period])], 
        [x for x in df_min_share_production.columns if "nemomod" in x]
    ] * 0.0
    


###
###    TEMPORARY: READ IN TRANSMISSION LOSSES BY COUNTRY
###

df_transmission_losses = pd.read_csv(
    sa.fp_csv_nemomod_transmission_losses
).drop(["year"], axis = 1)
df_transmission_losses = df_transmission_losses[
    df_transmission_losses[attr_time_period.key].isin(attr_time_period.key_values)
].reset_index(drop = True)



###
###    TEMPORARY: READ IN FUEL COSTS
###

df_fuel_costs = pd.read_csv(sa.fp_csv_nemomod_fuel_costs)



###
###    TEMPORARY: GET TECHNOLOGY MAX CAPACITY INVESTMENTS
###

fields_overwrite = sa.model_attributes.build_varlist(
    sa.model_attributes.subsec_name_entc,
    model_elec.modvar_entc_nemomod_total_annual_max_capacity_investment
)
df_max_cap = df_fake_data[[field_time_period] + fields_overwrite]



###
###  TEMPORARY OVERWRITE OF WOOD FUEL FRACTIONS FROM RECYCLED WOOD 
###


modvars_frac = model_energy.modvars_inen_list_fuel_fraction
dict_cp_wood = {}

# set cateogories
attr_ind = sa.model_attributes.dict_attributes.get("cat_industry")
dict_ind_to_recycling_target = attr_ind.field_maps.get("cat_industry_to_target_cat_industry_to_adjust_with_recycling")

# get target category (wood) and source from recycling
cat_target = model_afolu.cat_ippu_wood
cat_source = None
for k in dict_ind_to_recycling_target.keys():
    cat_source = k if (dict_ind_to_recycling_target.get(k) == f"``{cat_target}``") else cat_source


if cat_source is not None:
    for modvar in modvars_frac:

        cats = sa.model_attributes.get_variable_categories(modvar)
        subsec = sa.model_attributes.get_variable_subsector(modvar)
        
        if (cat_source in cats) and (cat_target in cats):

            field_source = sa.model_attributes.build_varlist(
                subsec,
                modvar,
                restrict_to_category_values = [cat_source]
            )[0]

            field_target = sa.model_attributes.build_varlist(
                subsec,
                modvar,
                restrict_to_category_values = [cat_target]
            )[0]

            dict_cp_wood.update({field_source: field_target})

            

######################
###                ###
###    END TEMP    ###
###                ###
######################

if True:
    # load in tables and make some quick modifications
    for k in dict_calibration_file_paths.keys():
        
        # read and clean columns
        df_read = pd.read_csv(dict_calibration_file_paths.get(k))
        dict_rnm = dict((x, x.lower()) for x in df_read.columns if x != x.lower())
        df_read.rename(columns = dict_rnm, inplace = True)
        
        # drop ISOs that are undefiuned
        df_read = df_read[df_read[field_iso].isin(all_iso)].reset_index(drop = True)
        
        ###
        ###  TEMPORARY: DROP OLD FUELS
        ###
        if True:
            fields_drop = [x for x in df_read.columns if ("gas_furnace" in x) or ("gas_petroleum_liquid" in x)]
            df_read.drop(fields_drop, axis = 1, inplace = True) if (len(fields_drop) > 0) else None
        
        
        ###
        ###  TEMPORARY: OVERWRITE AFOLU CLIMATE DATA
        ###
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_climate,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        
        
        ###
        ###    TEMPORARY: OVERWRITE OTHER KEY ENERGY DATA
        ###
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_overwrite,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
  
            
        
           
        ###
        ###    TEMPORARY: OVERWRITE FUEL COSTS
        ###
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_fuel_costs,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
            
            
        
        ###
        ###    TEMPORARY: OVERWRITE MAX CAPACITY
        ###
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_max_cap,
                fields_index = [sa.model_attributes.dim_time_period],
                overwrite_only = False
            )
            
            
        
            
            
        ###
        ###    TEMPORARY: RENAME FROM CALIBRATED FILE
        ###
        
        if False:
            dict_repl = {
                "gas_petroleum_liquid": "hydrocarbon_gas_liquids",
                "gas_furnace": "furnace_gas"
            }

            dict_rnm = {}

            for substr in dict_repl.keys():
                dict_rnm_cur = dict(
                    (x, x.replace(substr, dict_repl.get(substr))) 
                    for x in df_read.columns if substr in x
                )

                dict_rnm.update(dict_rnm_cur)

            df_read.rename(columns = dict_rnm, inplace = True)

        
        ####
        ####   TEMPORARY OVERWRITE OF RESIDUAL CAPACITY
        ####

        print("TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL CAPACITIES ARE FIXED")
        
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_residual_capacity_overwrite,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
            
        
        ####
        ####   TEMPORARY OVERWRITE OF ELECTRICAL TRANSMISSION LOSSES
        ####

        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_transmission_losses,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        

        ####
        ####   TEMPORARY OVERWRITE OF INITIAL PRODUCTION
        ####
        if False:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_prodvalues,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        

        ###
        ###    TEMPORARY: OVERWRITE EFFICIENCY FACTORS FOR RENEWABLES IN ENTC
        ###
        
        if True:
            fields = sa.model_attributes.build_varlist(
                "Energy Technology",
                "Technology Efficiency of Fuel Use",
                restrict_to_category_values = ["pp_hydropower", "pp_geothermal", "pp_solar", "pp_wind", "pp_ocean"]
            )
            df_read[fields] = 1
        

        ###
        ###    TEMPORARY: OVERWRITE EFFICIENCY FACTORS FOR ELECTRICITY HEAT IN SCOE/INEN
        ###
        
        if True:
            df_effic_factors_merge = sf.explode_merge(
                df_effic_factors,
                df_read[[field_iso]].drop_duplicates()
            )
            df_read = sf.match_df_to_target_df(
                df_read,
                df_effic_factors_merge,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        

        ###
        ###    TEMPORARY: ADD HYDROPOWER GENERATION CONSTRIANT
        ###

        if False:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_hydro_cap_synthetic,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        

        ###
        ###    TEMPORARY: OVERWRITE MinShareProduction for Baseline
        ###

        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_min_share_production,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
            #print(df_read[df_read[field_iso].isin(["BRA"])][[x for x in df_read.columns if "min_share_prod" in x]].head())


        # filter time periods and do some field cleaning
        if field_time_period not in df_read.columns:
            df_read[field_time_period] = np.array(np.array(df_read[field_year]) - 2015).astype(int)
            df_read = df_read[
                df_read[field_time_period].isin(attr_time_period.key_values)
            ].reset_index(drop = True)
        df_read = df_read[df_read[field_time_period] >= 0].reset_index(drop = True)
        df_read[field_region] = [dict_replace_iso.get(x.lower()) for x in list(df_read[field_iso])]

        # fields missing from input file: take from fake data
        fields_missing = list(set(df_fake_data.columns) - set(df_read.columns))
        fields_eliminate = list((set(df_read.columns) - set(df_fake_data.columns)) - set(fields_drop) - set(["strategy_id"]))

        df_read = pd.merge(
            df_read, 
            df_fake_data[[field_time_period] + fields_missing],
            on = [field_time_period],
            how = "left"
        )
        
        
        ###
        ###    TEMPORARY: OVERWRITE VALUES WITH FIXED VALUES SET IN DICTIONARY
        ###
        
        if True:
            dict_fields_to_value = dict((x, 1.0) for x in df_read.columns if x.startswith("demscalar"))
            
            for fld in dict_fields_to_value.keys():
                val = dict_fields_to_value.get(fld)
                df_read[fld] = val
                
                
        
        ###
        ###  TEMPORARY OVERWRITE OF WOOD FUEL FRACTIONS FROM RECYCLED WOOD 
        ###

        if False:
            for field_old in dict_cp_wood.keys():
                field_new = dict_cp_wood.get(field_old)
                df_read[field_new] = list(df_read[field_old])
                


        # filter out regions with invalid time period specifications
        df_read = check_time_periods(
            df_read,
            attr_time_period,
            [field_iso]
        )   
        # update available iso codes
        dict_isos.update({k: set(df_read[field_iso])})


        dict_calibration_tables.update({k: df_read})
        set_merge = set(df_read[field_region])
        dict_sets.update({k: set_merge})
        all_regions = set_merge if (all_regions is None) else (all_regions & set_merge)
        
        
        # write file for templatization input
        fp_write = os.path.join(dir_template_inputs, f"template_input_{k}.csv")
        dict_template_file_paths.update({k: fp_write})
        
        print(f"Writing sector '{k}' to {fp_write}...")
        dict_calibration_tables.get(k).to_csv(
            fp_write,
            index = None,
            encoding = "UTF-8"
        )
        print("Done.\n\n")

    attr_region = sa.model_attributes.dict_attributes.get("region")
    all_regions = sorted(list(set(attr_region.key_values) & all_regions))





TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL CAPACITIES ARE FIXED
Writing sector 'af' to /Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo/template_inputs/template_input_af.csv...
Done.


TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL CAPACITIES ARE FIXED
Writing sector 'ce' to /Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo/template_inputs/template_input_ce.csv...
Done.


TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL CAPACITIES ARE FIXED
Writing sector 'en' to /Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo/template_inputs/template_input_en.csv...
Done.


TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL CAPACITIES ARE FIXED
Writing sector 'ip' to /Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo/template_inputs/template_input_ip.csv...
Done.


TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL 

In [31]:
df = dict_calibration_tables.get("en")
df = pd.read_csv(dict_calibration_file_paths.get("en"))

def tmp_func(df):
    df = df[df[field_iso].isin(["BRA"])]
    ser = sa.model_attributes.get_standard_variables(
        df,
        "Private and Public Transportation Mode Share"
    ).iloc[0]
    
    print(ser)


In [53]:
importlib.reload(ma)
importlib.reload(sa)
importlib.reload(adt)
importlib.reload(ml)
importlib.reload(me)
warnings.filterwarnings("ignore")

#######################################
###                                 ###
###    IMPLEMENT TRANSFORMATIONS    ###
###                                 ###
#######################################




##  INITIALIZE SOME KEY ELEMENTS




# other input components
cats_renewable = [
    "pp_geothermal",
    "pp_hydropower",
    "pp_ocean",
    "pp_solar",
    "pp_wind"
]


# function for ramping down a cap based on the renewable energy target  
def get_vir_max_capacity(
    vec_implementation_ramp: np.ndarray,
    max_frac: float,
    delta_frac: float,
    dict_values_to_inds: Union[Dict, None] = None

) -> np.ndarray:
    """
    Buil a new value for the max_capacity based on vec_implementation_ramp.
        Starts with max_frac of a technicology's maximum residual capacity
        in the first period when vec_implementation_ramp != 0, then declines
        by delta_frac the specified number of time periods.

    Function Arguments
    ------------------
    - vec_implementation_ramp: vector of lever implementation ramp to use as
        reference
    - max_frac: fraction of maximum residual capacity to use as cap in first
        time period where vec_implementation_ramp > 0
    - delta_frac: delta to apply at each time period after the first time
        non-0 vec_implementation_ramp time_period.

    Keyword Arguments
    -----------------
    - dict_values_to_inds: optional dictionary mapping a value to row indicies
        to pass the value to. Can be used, for example, to provide a cap on new
        investments in early time periods. 
    """

    vec_implementation_ramp_max_capacity = np.ones(len(vec_implementation_ramp))
    i0 = None

    for i in range(len(vec_implementation_ramp)):
        if vec_implementation_ramp[i] == 0:
            vec_implementation_ramp_max_capacity[i] = -999
        else:
            i0 = i if (i0 is None) else i0

            vec_implementation_ramp_max_capacity[i] = max(max_frac - delta_frac*(i - i0), 0.0)


    if isinstance(dict_values_to_inds, dict):
        for k in dict_values_to_inds.keys():
            np.put(vec_implementation_ramp_max_capacity, dict_values_to_inds.get(k), k)

    return vec_implementation_ramp_max_capacity


n_tp = len(sa.model_attributes.dict_attributes.get("dim_time_period").key_values)
vec_implementation_ramp = np.array([max(0, min((x - 10)/25, 1)) for x in range(n_tp)]) # start in 2025 (last 0) and finish in 2050 (1)
vec_implementation_ramp_renewable_cap = get_vir_max_capacity(vec_implementation_ramp, 0.05, 0.01)


##  INITIALIZE SOME SHARED COMPONENTS

# information for renewable energy target transformation
# ramp down for MSP of key techs
vec_implementation_ramp_renewable_cap = get_vir_max_capacity(
    vec_implementation_ramp, 
    0.05, 
    0.01
    #dict_values_to_inds = {0: [0, 1, 2, 3, 4, 5]}
)

vec_implementation_ramp_renewable_cap = get_vir_max_capacity(vec_implementation_ramp, 0.05, 0.01)
dict_entc_renewable_target_ramp = {
     "vec": vec_implementation_ramp_renewable_cap.copy(),
     "type": "scalar"
}

# direct inputs to transformation 
#  set dict_entc_renewable_target_cats_max_investment = None to turn off
#  set dict_entc_renewable_target_msp to None to turn off
dict_entc_renewable_target_cats_max_investment = {
    "pp_hydropower": dict_entc_renewable_target_ramp,
    "pp_nuclear": dict_entc_renewable_target_ramp
}
dict_entc_renewable_target_msp = {
    "pp_solar": 0.15,
    "pp_geothermal": 0.1,
    "pp_wind": 0.15
}



# initialze output components

df_input = dict_calibration_tables.get("en").copy()
df_prepend = sf.add_data_frame_fields_from_dict(
    df_input,
    {
        sa.model_attributes.dim_strategy_id: 0
    },
    prepend_q = True,
    overwrite_fields = True
)
df_all_transformations = [df_prepend]
dict_strat_name_to_ind = {}
ind_strategy = 0


################################
#    SETUP SHARED FUNCTIONS    #
################################

# shared function for implementing "green energy"
def transformation_clean_grid(
    df_entc: pd.DataFrame,
    strat: int,
    include_hydrogen = True
) -> pd.DataFrame:
    """
    Implement the "clean grid" transformation (shared repeatability),
        which includes 95% renewable energy target and green hydrogen
    """
    # ENTC: 95% of today's fossil-fuel electricity is generated by renewables in 2050
    df_strat_cur = adt.transformation_entc_renewable_target(
        df_entc,
        0.95,
        cats_renewable,
        vec_implementation_ramp,
        sa.model_attributes,
        model_elec,
        dict_cats_entc_max_investment = dict_entc_renewable_target_cats_max_investment,
        magnitude_renewables = dict_entc_renewable_target_msp,
        model_energy = model_energy,
        strategy_id = strat
    )

    if include_hydrogen:
        # ENTC: add green hydrogen
        df_strat_cur = adt.transformation_entc_hydrogen_electrolysis(
            df_strat_cur,
            0.95,
            vec_implementation_ramp,
            sa.model_attributes,
            model_elec,
            model_energy = model_energy,
            strategy_id = strat
        )

    return df_strat_cur




##############################
###                        ###
###    BEGIN STRATEGIES    ###
###                        ###
##############################

##  MINIMIZE LEAKS STRATEGY (FGTV)

strat_name = "FGTV: Minimize leaks"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_fgtv_reduce_leaks(
        df_input,
        0.8, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")

    
    

##  MAXIMIZE FLARING STRATEGY (FGTV)

strat_name = "FGTV: Maximize flaring"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_fgtv_maximize_flaring(
        df_input,
        0.8, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")


    

##  FGTV BUNDLE

strat_name = "FGTV: All Fugitive Emissions Transformations"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

# bundle in a function that can be passed in other strategies
def all_transformations_fgtv(
    df_fgtv: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement all fugitive emission transformations on data frame df_fgtv.
    """
    # FGTV: MINIMIZE LEAKS STRATEGY (FGTV)
    df_strat_cur = adt.transformation_fgtv_reduce_leaks(
        df_fgtv,
        0.8, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    # FGTV: MAXIMIZE FLARING STRATEGY (FGTV)
    df_strat_cur = adt.transformation_fgtv_maximize_flaring(
        df_strat_cur,
        0.8, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    return df_strat_cur

# apply if strategy is valid
if strat is not None:
    df_all_transformations[ind_strategy] = all_transformations_fgtv(df_input, strat)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")





###########################
#    INDUSTRIAL ENERGY    #
###########################

# shared INEN parameters
attr_ippu = sa.model_attributes.dict_attributes.get("cat_industry")
cats_inen_high_heat = ["cement", "chemicals", "glass", "lime_and_carbonite", "metals"]
attr_ippu = sa.model_attributes.dict_attributes.get("cat_industry")
modvars_inen_fuel_switching = adt.transformation_inen_shift_modvars(
    df_input,
    None,
    None,
    sa.model_attributes,
    return_modvars_only = True
)
cats_inen_fuel_switching = set({})
for modvar in modvars_inen_fuel_switching:
    cats_inen_fuel_switching = cats_inen_fuel_switching | set(
        sa.model_attributes.get_variable_categories(modvar)
    )
cats_inen_not_high_heat = sorted(list(cats_inen_fuel_switching - set(cats_inen_high_heat))) 

frac_inen_low_temp_elec = 0.95*0.45
frac_inen_high_temp_elec_hydg = 0.5*0.45 # assume 70% high heat
frac_inen_shift_denom = frac_inen_low_temp_elec + 2*frac_inen_high_temp_elec_hydg




##  MAXIMIZE INDUSTRIAL ENERGY EFFICIENCY (INEN)

strat_name = "INEN: Maximize industrial energy efficiency"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_inen_maximize_energy_efficiency(
        df_input,
        0.3, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  INEN: MAXIMIZE INDUSTRIAL PRODUCTION EFFICIENCY (INEN)

strat_name = "INEN: Maximize industrial production efficiency"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_inen_maximize_production_efficiency(
        df_input,
        0.4, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  INEN: FUEL SWITCH LOW-TEMP THERMAL PROCESSES

strat_name = "INEN: Fuel switch low-temp thermal processes to industrial heat pumps"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_inen_shift_modvars(
        df_input,
        frac_inen_low_temp_elec,#0.4275, 
        vec_implementation_ramp,
        sa.model_attributes,
        dict_modvar_specs = {
            model_energy.modvar_inen_frac_en_electricity: 1.0
        },
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  INEN: FUEL SWITCH LOW-TEMP THERMAL PROCESSES w/RENEABLES

strat_name = "INEN: Fuel switch low-temp thermal processes to industrial heat pumps with Renewable Grid"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_strat_cur = adt.transformation_inen_shift_modvars(
        df_input,
        frac_inen_low_temp_elec,#0.4275, 
        vec_implementation_ramp,
        sa.model_attributes,
        dict_modvar_specs = {
            model_energy.modvar_inen_frac_en_electricity: 1.0
        },
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    # ENTC: 95% of today's fossil-fuel electricity is generated by renewables in 2050
    df_strat_cur = adt.transformation_entc_renewable_target(
        df_strat_cur,
        0.95,
        cats_renewable,
        vec_implementation_ramp,
        sa.model_attributes,
        model_elec,
        dict_cats_entc_max_investment = dict_entc_renewable_target_cats_max_investment,
        magnitude_renewables = dict_entc_renewable_target_msp,
        model_energy = model_energy,
        strategy_id = strat
    )

    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  INEN: FUEL SWITCH HIGH-TEMP THERMAL PROCESSES

strat_name = "INEN: Fuel switch medium and high-temp thermal processes to hydrogen and electricity"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_inen_shift_modvars(
        df_input,
        2*frac_inen_high_temp_elec_hydg, # 90% of 50%
        vec_implementation_ramp,
        sa.model_attributes,
        categories = cats_inen_high_heat,
        dict_modvar_specs = {
            model_energy.modvar_inen_frac_en_electricity: 0.5,
            model_energy.modvar_inen_frac_en_hydrogen: 0.5,
        },
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  INEN: FUEL SWITCH HIGH-TEMP THERMAL PROCESSES w/RENEWABLES

strat_name = "INEN: Fuel switch medium and high-temp thermal processes to hydrogen and electricity with Renewable Grid and Green Hydrogen"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_strat_cur = adt.transformation_inen_shift_modvars(
        df_input,
        2*frac_inen_high_temp_elec_hydg, # 90% of 50%
        vec_implementation_ramp,
        sa.model_attributes,
        categories = cats_inen_high_heat,
        dict_modvar_specs = {
            model_energy.modvar_inen_frac_en_electricity: 0.5,
            model_energy.modvar_inen_frac_en_hydrogen: 0.5,
        },
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    # ENTC: 95% of today's fossil-fuel electricity is generated by renewables in 2050
    df_strat_cur = adt.transformation_entc_renewable_target(
        df_strat_cur,
        0.95,
        cats_renewable,
        vec_implementation_ramp,
        sa.model_attributes,
        model_elec,
        dict_cats_entc_max_investment = dict_entc_renewable_target_cats_max_investment,
        magnitude_renewables = dict_entc_renewable_target_msp,
        model_energy = model_energy,
        strategy_id = strat
    )

    # ENTC: add green hydrogen
    df_strat_cur = adt.transformation_entc_hydrogen_electrolysis(
        df_strat_cur,
        0.95,
        vec_implementation_ramp,
        sa.model_attributes,
        model_elec,
        model_energy = model_energy,
        strategy_id = strat
    )

    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  INEN BUNDLE

strat_name = "INEN: All Industrial Energy Transformations"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

# bundle in a function that can be passed in other strategies
def all_transformations_inen(
    df_inen: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement all industrial energy transformations on data frame df_inen.
    """

    # INEN: Maximize industrial energy efficiency
    df_strat_cur = adt.transformation_inen_maximize_energy_efficiency(
        df_inen,
        0.3, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    # INEN: MAXIMIZE INDUSTRIAL PRODUCTION EFFICIENCY (INEN)
    df_strat_cur = adt.transformation_inen_maximize_production_efficiency(
        df_strat_cur,
        0.3, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    # INEN: FUEL SWITCH HIGH-TEMP THERMAL PROCESSES + Fuel switch low-temp thermal processes to industrial heat pumps (ONLY FOR HIGH HEAT CATS)
    df_strat_cur = adt.transformation_inen_shift_modvars(
        df_strat_cur,
        frac_inen_shift_denom, # 90% of 50% 22.5 + 45 = 67.5% + 22.5% hydrogen 4.5/18.5 9/37 
        vec_implementation_ramp, 
        sa.model_attributes,
        categories = cats_inen_high_heat,
        dict_modvar_specs = {
            model_energy.modvar_inen_frac_en_electricity: (frac_inen_low_temp_elec + frac_inen_high_temp_elec_hydg)/frac_inen_shift_denom,
            model_energy.modvar_inen_frac_en_hydrogen: frac_inen_high_temp_elec_hydg/frac_inen_shift_denom,
        },
        model_energy = model_energy,
        strategy_id = strat
    )

    # INEN: FUEL SWITCH LOW-TEMP THERMAL PROCESSES TO INDUSTIAL HEAT PUMPS (FOR NON-HIGH HEAT CATEGORIES)
    df_strat_cur = adt.transformation_inen_shift_modvars(
        df_strat_cur,
        frac_inen_low_temp_elec,
        vec_implementation_ramp,
        sa.model_attributes,
        categories = cats_inen_not_high_heat,
        dict_modvar_specs = {
            model_energy.modvar_inen_frac_en_electricity: 1.0
        },
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    return df_strat_cur

# apply function if strategy is valid
if strat is not None:
    df_all_transformations[ind_strategy] = all_transformations_inen(df_input, strat)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  INEN BUNDLE w/RENEWABLES

strat_name = "INEN: All Industrial Energy Transformations with Renewable Grid and Green Hydrogen"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    # apply all inen + clean grid
    df_strat_cur = all_transformations_inen(df_input, strat)
    df_strat_cur = transformation_clean_grid(df_strat_cur, strat)

    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")


    
    
    


########################
#    TRANSPORTATION    #
########################


##  ELECTRIFY LIGHT DUTY ROAD TRANSPORT

strat_name = "TRNS: Electrify light duty road transport"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

def electricy_light_duty_road_trns(
    df_trns: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement the electrify light duty road transformation
    """
    df_out = adt.transformation_trns_fuel_shift_to_target(
        df_trns,
        0.7,
        vec_implementation_ramp,
        sa.model_attributes,
        dict_modvar_specs = {
            model_energy.modvar_trns_fuel_fraction_electricity: 1.0
        },
        categories = ["road_light"],
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )
    
    return df_out


if strat is not None:
    df_all_transformations[ind_strategy] = electricy_light_duty_road_trns(df_input, strat)    
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  ELECTRIFY MEDIUM-DUTY ROAD TRANSPORT

strat_name = "TRNS: Electrify medium duty road transport"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

def electricy_medium_duty_road_trns(
    df_trns: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement the electrify rail medium duty road transport
    """
    df_out = adt.transformation_trns_fuel_shift_to_target(
        df_trns,
        0.5,
        vec_implementation_ramp,
        sa.model_attributes,
        dict_modvar_specs = {
            model_energy.modvar_trns_fuel_fraction_electricity: 1.0
        },
        categories = ["road_heavy_freight", "road_heavy_regional", "public"],
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )
    
    return df_out


if strat is not None:
    df_all_transformations[ind_strategy] = electricy_medium_duty_road_trns(df_input, strat)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  ELECTRIFY RAIL

strat_name = "TRNS: Electrify rail"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

def electricy_rail_trns(
    df_trns: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement the electrify rail transformation
    """
    df_out = adt.transformation_trns_fuel_shift_to_target(
        df_trns,
        0.25,
        vec_implementation_ramp,
        sa.model_attributes,
        dict_modvar_specs = {
            model_energy.modvar_trns_fuel_fraction_electricity: 1.0
        },
        categories = ["rail_freight", "rail_passenger"],
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )
    
    return df_out
    
    
if strat is not None:
    df_all_transformations[ind_strategy] = electricy_rail_trns(df_input, strat)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")


    
    
##  FUEL SWITCH BUNDLE

strat_name = "TRNS: Fuel switch bundle"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_strat_cur = electricy_light_duty_road_trns(df_input, strat) 
    df_strat_cur = electricy_medium_duty_road_trns(df_strat_cur, strat)
    df_strat_cur = electricy_rail_trns(df_strat_cur, strat)
    
    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")
    
    
    

##  INCREASE TRANSPORTATION ELECTRICAL ENERGY EFFICIENCY

strat_name = "TRNS: Increase transportation electricity energy efficiency"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

def increase_efficiency_elec_trns(
    df_trns: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement the Increase transportation electricity energy efficiency 
        transformation
    """
    df_out = adt.transformation_trns_increase_energy_efficiency_electric(
        df_trns,
        0.25, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )
    
    return df_out


if strat is not None:
    df_all_transformations[ind_strategy] = increase_efficiency_elec_trns(df_input, strat)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")


    
    
##  INCREASE TRANSPORTATION NON-ELECTRICITY EFFICIENCY

strat_name = "TRNS: Increase transportation non-electricity energy efficiency"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

def increase_efficiency_nonelec_trns(
    df_trns: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement the Increase transportation non-electricity energy efficiency 
        transformation
    """
    df_out = adt.transformation_trns_increase_energy_efficiency_non_electric(
        df_trns,
        0.25, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )
    
    return df_out


if strat is not None:
    df_all_transformations[ind_strategy] = increase_efficiency_nonelec_trns(df_input, strat)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")

    
    
    
##  EFFICIENCY BUNDLE

strat_name = "TRNS: Efficiency bundle"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_strat_cur = increase_efficiency_elec_trns(df_input, strat) 
    df_strat_cur = increase_efficiency_nonelec_trns(df_strat_cur, strat)
    
    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")



    
##  REDUCE DEMAND FOR TRANSPORT

strat_name = "TRNS: Reduce demand for transport"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

def reduce_demand_trns(
    df_trns: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement the Reduce demand for transport transformation
    """
    df_out = adt.transformation_trde_reduce_demand(
        df_trns,
        0.25, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )
    
    return df_out


if strat is not None:
    df_all_transformations[ind_strategy] = reduce_demand_trns(df_input, strat)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")



    
##  INCREASE VEHICLE OCCUPANCY

strat_name = "TRNS: Increase occupancy for private vehicles"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

def increase_vehicle_occupancy_trns(
    df_trns: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement the Reduce demand for transport transformation
    """
    df_out = adt.transformation_trns_increase_vehicle_occupancy(
        df_trns,
        0.25, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )
    
    return df_out


if strat is not None:
    df_all_transformations[ind_strategy] = increase_vehicle_occupancy_trns(df_input, strat)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")

    
    
    
##  DEMAND MANAGEMENT BUNDLE

strat_name = "TRNS: Demand management bundle"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_strat_cur = reduce_demand_trns(df_input, strat) 
    df_strat_cur = increase_vehicle_occupancy_trns(df_strat_cur, strat)
    
    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")


    
    
##  MODE SHIFT PASSENGER VEHICLES

strat_name = "TRNS: Mode shift passenger vehicles to others"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

def mode_shift_passenger_trns(
    df_trns: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement the Mode shift passenger vehicles to others transformation
    """
    df_out = adt.transformation_general(
        df_trns,
        sa.model_attributes,
        {
            model_energy.modvar_trns_modeshare_public_private: {
                "bounds": (0, 1),
                "magnitude": 0.3,
                "magnitude_type": "transfer_value",
                "categories_source": ["road_light"],
                "categories_target": {
                    "human_powered": (1/6),
                    "powered_bikes": (2/6),
                    "public": 0.5
                },
                "vec_ramp": vec_implementation_ramp
            }
        },
        model_energy = model_energy,
        strategy_id = strat
    )
    
    return df_out


if strat is not None:
    df_all_transformations[ind_strategy] = mode_shift_passenger_trns(df_input, strat)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  MODE SHIFT REGIONAL PASSENGER TRAVEL

strat_name = "TRNS: Mode shift regional passenger travel"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

def mode_shift_regional_trns(
    df_trns: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement the Mode shift regional passenger vehicles to others 
        transformation
    """
    df_out = adt.transformation_general(
        df_trns,
        sa.model_attributes,
        {
            model_energy.modvar_trns_modeshare_regional: {
                "bounds": (0, 1),
                "magnitude": 0.25,
                "magnitude_type": "transfer_value",
                "categories_source": ["aviation"],
                "categories_target": {
                    "rail_passenger": 0.5,
                    "road_heavy_regional": 0.5
                },
                "vec_ramp": vec_implementation_ramp
            }
        },
        model_energy = model_energy,
        strategy_id = strat
    )
    
    return df_out


if strat is not None:
    df_all_transformations[ind_strategy] = mode_shift_regional_trns(df_input, strat)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  MODE SHIFT FREIGHT

strat_name = "TRNS: Mode shift freight"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

def mode_shift_freight_trns(
    df_trns: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement the Mode shift freight vehicles to others 
        transformation
    """
    df_out = adt.transformation_general(
        df_trns,
        sa.model_attributes,
        {
            model_energy.modvar_trns_modeshare_freight: {
                "bounds": (0, 1),
                "magnitude": 0.2,
                "magnitude_type": "transfer_value",
                "categories_source": ["road_heavy_freight"],
                "categories_target": {
                    "rail_freight": 1.0
                },
                "vec_ramp": vec_implementation_ramp
            }
        },
        model_energy = model_energy,
        strategy_id = strat
    )
    
    return df_out


if strat is not None:
    df_all_transformations[ind_strategy] = mode_shift_freight_trns(df_input, strat)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")


    
    
##  MODE SHIFT BUNDLE

strat_name = "TRNS: Mode shift bundle"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_strat_cur = mode_shift_passenger_trns(df_input, strat) 
    df_strat_cur = mode_shift_regional_trns(df_strat_cur, strat)
    df_strat_cur = mode_shift_freight_trns(df_strat_cur, strat)
    
    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")
    



##  TRNS BUNDLE 

strat_name = "TRNS: All Transportation Transformations"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)


# bundle in a function that can be passed in other strategies
def all_transformations_trns(
    df_trns: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement all transportation transformations on data frame df_trns.
    """
    ##  ELECTRIFY LIGHT DUTY ROAD TRANSPORT
    df_strat_cur = electricy_light_duty_road_trns(df_trns, strat) 
    ##  ELECTRIFY MEDIUM-DUTY ROAD TRANSPORT
    df_strat_cur = electricy_medium_duty_road_trns(df_strat_cur, strat)
    ##  ELECTRICITY RAIL
    df_strat_cur = electricy_rail_trns(df_strat_cur, strat)
    ##  INCREASE TRANSPORTATION ELECTRICAL ENERGY EFFICIENCY
    df_strat_cur = increase_efficiency_elec_trns(df_strat_cur, strat) 
    ##  INCREASE TRANSPORTATION NON-ELECTRICITY EFFICIENCY
    df_strat_cur = increase_efficiency_nonelec_trns(df_strat_cur, strat)
    ##  REDUCE DEMAND FOR TRANSPORT
    df_strat_cur = reduce_demand_trns(df_strat_cur, strat) 
    ##  INCREASE VEHICLE OCCUPANCY
    df_strat_cur = increase_vehicle_occupancy_trns(df_strat_cur, strat) 
    ##  MODE SHIFT PASSENGER VEHICLES
    df_strat_cur = mode_shift_passenger_trns(df_input, strat) 
    ##  MODE SHIFT REGIONAL PASSENGER TRAVEL
    df_strat_cur = mode_shift_regional_trns(df_strat_cur, strat)
    ##  MODE SHIFT FREIGHT
    df_strat_cur = mode_shift_freight_trns(df_strat_cur, strat)

    return df_strat_cur


if strat is not None:
    df_all_transformations[ind_strategy] = all_transformations_trns(df_input, strat)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  TRNS BUNDLE w/RENEWABLES

strat_name = "TRNS: All Transportation Transformations with Renewable Grid and Green Hydrogen"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_strat_cur = all_transformations_trns(df_input, strat)
    df_strat_cur = transformation_clean_grid(df_strat_cur, strat)

    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")







##############################
#    SCOE TRANSFORMATIONS    #
##############################


##  SCOE Reduce demand for heat energy

strat_name = "SCOE: Reduce end-use demand for heat energy by improving building shell"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_scoe_reduce_demand_for_heat_energy(
        df_input,
        0.5,
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")

    


##  SCOE Switch to electricity for heat

strat_name = "SCOE: Switch to electricity for heat using heat pumps, electric stoves, etc."
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_scoe_electrify_category_to_target(
        df_input,
        0.95,
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")


    

##  SCOE Switch to electricity for heat w/RENEWABLE

strat_name = "SCOE: Switch to electricity for heat using heat pumps, electric stoves, etc. with Renewable Grid"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_strat_cur = adt.transformation_scoe_electrify_category_to_target(
        df_input,
        0.95,
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    # ENTC: 95% of today's fossil-fuel electricity is generated by renewables in 2050
    df_strat_cur = adt.transformation_entc_renewable_target(
        df_strat_cur,
        0.95,
        cats_renewable,
        vec_implementation_ramp,
        sa.model_attributes,
        model_elec,
        dict_cats_entc_max_investment = dict_entc_renewable_target_cats_max_investment,
        magnitude_renewables = dict_entc_renewable_target_msp,
        model_energy = model_energy,
        strategy_id = strat
    )

    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")


    

##  SCOE INCREASE APPLIANCE EFFICIENCY

strat_name = "SCOE: Increase appliance efficiency"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_scoe_reduce_demand_for_appliance_energy(
        df_input,
        0.5,
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")

    


##  SCOE BUNDLE

strat_name = "SCOE: All Stationary Combustion Transformations"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)


# bundle in a function that can be passed in other strategies
def all_transformations_scoe(
    df_scoe: pd.DataFrame,
    strat: int
) -> pd.DataFrame:
    """
    Implement all statioary combustion and other energy transformations 
        on data frame df_scoe.
    """
    # SCOE Reduce demand for heat energy
    df_strat_cur = adt.transformation_scoe_reduce_demand_for_heat_energy(
        df_scoe,
        0.5,
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    # SCOE Switch to electricity for heat
    df_strat_cur = adt.transformation_scoe_electrify_category_to_target(
        df_strat_cur,
        0.95,
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    # SCOE INCREASE APPLIANCE EFFICIENCY
    df_strat_cur = adt.transformation_scoe_reduce_demand_for_appliance_energy(
        df_strat_cur,
        0.5,
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    return df_strat_cur

# apply if properly specified 
if strat is not None:
    df_all_transformations[ind_strategy] = all_transformations_scoe(df_input, strat)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  SCOE BUNDLE w/Reneawble

strat_name = "SCOE: All Stationary Combustion Transformations with Renewable Grid"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    # SCOE Reduce demand for heat energy
    df_strat_cur = all_transformations_scoe(df_input, strat)
    df_strat_cur = transformation_clean_grid(df_strat_cur, strat, include_hydrogen = False)

    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




#############################################################
#    TECHNOLOGY (ELECTRICITY GENERATION) TRANSFORMATIONS    #
#############################################################

##  ENTC Retire fossil fuel plants early

strat_name = "EN - Retire fossil fuel plants early"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:

    vec_ramp_coal = np.array([max(0, min((x - 10)/5, 1)) for x in range(n_tp)])
    vec_ramp_gas = np.array([max(0, min((x - 10)/15, 1)) for x in range(n_tp)])

    df_all_transformations[ind_strategy] = adt.transformation_entc_retire_fossil_fuel_early(
        df_input,
        {
            "pp_coal": vec_ramp_coal,
            "pp_gas": vec_ramp_gas,
            "pp_oil": vec_ramp_coal
        },
        sa.model_attributes,
        model_elec,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  NOT IN attribute_dim_strategy_id. ENTC Increase Renewables from Baseline

strat_name = "EN - Increase renewables"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_entc_increase_renewables(
        df_input,
        2.0,
        vec_implementation_ramp,
        sa.model_attributes,
        model_elec,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  NOT IN attribute_dim_strategy_id. ENTC Reduce cost of renewables

strat_name = "EN - Reduce cost of renewables"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_entc_reduce_cost_of_renewables(
        df_input,
        0.5,
        vec_implementation_ramp,
        sa.model_attributes,
        model_elec,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  NOT IN attribute_dim_strategy_id. ENTC Increase efficiency of electricity production

strat_name = "EN - Increase efficiency of electricity production"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_entc_increase_efficiency_of_electricity_production(
        df_input,
        0.25,
        vec_implementation_ramp,
        sa.model_attributes,
        model_elec,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  Reduce transmission losses

strat_name = "ENTC: Reduce transmission losses"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_entc_specify_transmission_losses(
        df_input,
        0.04,
        vec_implementation_ramp,
        sa.model_attributes,
        model_elec,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  ENTC: 95% OF ELECTRICITY GENERATED BY RENEWABLES IN 2050

strat_name = "ENTC: 95% of electricity is generated by renewables in 2050"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = transformation_clean_grid(df_input, strat, include_hydrogen = False)
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  ENTC: 95% OF ELECTRICITY GENERATED BY RENEWABLES IN 2050 + Transmission Loss reduction

strat_name = "ENTC: 95% of electricity is generated by renewables in 2050 with Reduced Transmission Losses"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:

    df_strat_cur = transformation_clean_grid(df_input, strat, include_hydrogen = False)

    # add in reduction in transmission losses
    df_strat_cur = adt.transformation_entc_specify_transmission_losses(
        df_strat_cur,
        0.04,
        vec_implementation_ramp,
        sa.model_attributes,
        model_elec,
        model_energy = model_energy,
        strategy_id = strat
    )

    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")



##  ENTC: Least cost solution

strat_name = "ENTC: Least cost solution"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_entc_least_cost_solution(
        df_input,
        vec_implementation_ramp,
        sa.model_attributes,
        model_electricity = model_elec,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##########################################################
#    CARBON CAPTURE AND SEQUESTRATION TRANSFORMATIONS    #
##########################################################

##  CSQ Fuel switch low-temp thermal processes



##  CCSQ Fuel switch high-temp thermal processes



##  CCSQ - Increase Direct Air Capture

strat_name = "CCSQ: Increase direct air capture"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:
    df_all_transformations[ind_strategy] = adt.transformation_ccsq_increase_direct_air_capture(
        df_input,
        50,
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




################################
#    COMBINATION STRATEGIES    #
################################

##  EFFICIENCY BUNDLE

strat_name = "EN: Efficiency Bundle"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None: 
    # INEN: Maximize industrial energy efficiency 
    df_strat_cur = adt.transformation_inen_maximize_energy_efficiency(
        df_input,
        0.3, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    # INEN: Fuel switch low-temp thermal processes to industrial heat pumps
    df_strat_cur = adt.transformation_inen_shift_modvars(
        df_strat_cur,
        frac_inen_low_temp_elec, # 50% of 90% (assume 10% is non-heat, so withcing)
        vec_implementation_ramp,
        sa.model_attributes,
        dict_modvar_specs = {
            model_energy.modvar_inen_frac_en_electricity: 1.0
        },
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    # SCOE: Reduce end-use demand for heat energy by improving building shell
    df_strat_cur = adt.transformation_scoe_reduce_demand_for_heat_energy(
        df_strat_cur,
        0.5,
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    # SCOE: Switch to electricity for heat using heat pumps, electric stoves, etc.
    df_strat_cur = adt.transformation_scoe_electrify_category_to_target(
        df_strat_cur,
        0.95,
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    # SCOE: Increase appliance efficiency
    df_strat_cur = adt.transformation_scoe_reduce_demand_for_appliance_energy(
        df_strat_cur,
        0.5,
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    ##  INCREASE TRANSPORTATION ELECTRICAL ENERGY EFFICIENCY
    df_strat_cur = adt.transformation_trns_increase_energy_efficiency_electric(
        df_strat_cur,
        0.25, 
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    ##  INCREASE TRANSPORTATION NON-ELECTRICITY EFFICIENCY 
    df_strat_cur = adt.transformation_trns_increase_energy_efficiency_non_electric(
            df_strat_cur,
            0.25, 
            vec_implementation_ramp,
            sa.model_attributes,
            model_energy = model_energy,
            strategy_id = strat
        )

    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  EN: Fuel Switch Bundle

strat_name = "EN: Fuel Switch Bundle"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:

    # INEN: FUEL SWITCH HIGH-TEMP THERMAL PROCESSES + Fuel switch low-temp thermal processes to industrial heat pumps (ONLY FOR HIGH HEAT CATS)
    df_strat_cur = adt.transformation_inen_shift_modvars(
        df_input,
        frac_inen_shift_denom, # 90% of 50% 22.5 + 45 = 67.5% + 22.5% hydrogen 4.5/18.5 9/37 
        vec_implementation_ramp, 
        sa.model_attributes,
        categories = cats_inen_high_heat,
        dict_modvar_specs = {
            model_energy.modvar_inen_frac_en_electricity: (frac_inen_low_temp_elec + frac_inen_high_temp_elec_hydg)/frac_inen_shift_denom,
            model_energy.modvar_inen_frac_en_hydrogen: frac_inen_high_temp_elec_hydg/frac_inen_shift_denom,
        },
        model_energy = model_energy,
        strategy_id = strat
    )

    # INEN: FUEL SWITCH LOW-TEMP THERMAL PROCESSES TO INDUSTIAL HEAT PUMPS (FOR NON-HIGH HEAT CATEGORIES)
    df_strat_cur = adt.transformation_inen_shift_modvars(
        df_strat_cur,
        frac_inen_low_temp_elec,
        vec_implementation_ramp,
        sa.model_attributes,
        categories = cats_inen_not_high_heat,
        dict_modvar_specs = {
            model_energy.modvar_inen_frac_en_electricity: 1.0
        },
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    # SCOE: Switch to electricity for heat using heat pumps, electric stoves, etc.
    df_strat_cur = adt.transformation_scoe_electrify_category_to_target(
        df_strat_cur,
        0.95,
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    ##  ELECTRIFY LIGHT DUTY ROAD TRANSPORT
    df_strat_cur = adt.transformation_trns_fuel_shift_to_target(
        df_strat_cur,
        0.7,
        vec_implementation_ramp,
        sa.model_attributes,
        dict_modvar_specs = {
            model_energy.modvar_trns_fuel_fraction_electricity: 1.0
        },
        categories = ["road_light"],
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    ##  ELECTRIFY MEDIUM-DUTY ROAD TRANSPORT
    df_strat_cur = adt.transformation_trns_fuel_shift_to_target(
        df_strat_cur,
        0.5,
        vec_implementation_ramp,
        sa.model_attributes,
        dict_modvar_specs = {
            model_energy.modvar_trns_fuel_fraction_electricity: 1.0
        },
        categories = ["road_heavy_freight", "road_heavy_regional", "public"],
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    ##  ELECTRICITY RAIL
    df_strat_cur = adt.transformation_trns_fuel_shift_to_target(
        df_strat_cur,
        0.25,
        vec_implementation_ramp,
        sa.model_attributes,
        dict_modvar_specs = {
            model_energy.modvar_trns_fuel_fraction_electricity: 1.0
        },
        categories = ["rail_freight", "rail_passenger"],
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")



##  EN: Fuel Switch Bundle w/RENEWABLES

strat_name = "EN: Fuel Switch Bundle with Renewable Grid and Green Hydrogen"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)

if strat is not None:

    # INEN: FUEL SWITCH HIGH-TEMP THERMAL PROCESSES + Fuel switch low-temp thermal processes to industrial heat pumps (ONLY FOR HIGH HEAT CATS)
    df_strat_cur = adt.transformation_inen_shift_modvars(
        df_input,
        frac_inen_shift_denom, # 90% of 50% 22.5 + 45 = 67.5% + 22.5% hydrogen 4.5/18.5 9/37 
        vec_implementation_ramp, 
        sa.model_attributes,
        categories = cats_inen_high_heat,
        dict_modvar_specs = {
            model_energy.modvar_inen_frac_en_electricity: (frac_inen_low_temp_elec + frac_inen_high_temp_elec_hydg)/frac_inen_shift_denom,
            model_energy.modvar_inen_frac_en_hydrogen: frac_inen_high_temp_elec_hydg/frac_inen_shift_denom,
        },
        model_energy = model_energy,
        strategy_id = strat
    )

    # INEN: FUEL SWITCH LOW-TEMP THERMAL PROCESSES TO INDUSTIAL HEAT PUMPS (FOR NON-HIGH HEAT CATEGORIES)
    df_strat_cur = adt.transformation_inen_shift_modvars(
        df_strat_cur,
        frac_inen_low_temp_elec,
        vec_implementation_ramp,
        sa.model_attributes,
        categories = cats_inen_not_high_heat,
        dict_modvar_specs = {
            model_energy.modvar_inen_frac_en_electricity: 1.0
        },
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    # SCOE: Switch to electricity for heat using heat pumps, electric stoves, etc.
    df_strat_cur = adt.transformation_scoe_electrify_category_to_target(
        df_strat_cur,
        0.95,
        vec_implementation_ramp,
        sa.model_attributes,
        model_energy = model_energy,
        strategy_id = strat
    )

    ##  ELECTRIFY LIGHT DUTY ROAD TRANSPORT
    df_strat_cur = adt.transformation_trns_fuel_shift_to_target(
        df_strat_cur,
        0.7,
        vec_implementation_ramp,
        sa.model_attributes,
        dict_modvar_specs = {
            model_energy.modvar_trns_fuel_fraction_electricity: 1.0
        },
        categories = ["road_light"],
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    ##  ELECTRIFY MEDIUM-DUTY ROAD TRANSPORT
    df_strat_cur = adt.transformation_trns_fuel_shift_to_target(
        df_strat_cur,
        0.5,
        vec_implementation_ramp,
        sa.model_attributes,
        dict_modvar_specs = {
            model_energy.modvar_trns_fuel_fraction_electricity: 1.0
        },
        categories = ["road_heavy_freight", "road_heavy_regional", "public"],
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    ##  ELECTRICITY RAIL
    df_strat_cur = adt.transformation_trns_fuel_shift_to_target(
        df_strat_cur,
        0.25,
        vec_implementation_ramp,
        sa.model_attributes,
        dict_modvar_specs = {
            model_energy.modvar_trns_fuel_fraction_electricity: 1.0
        },
        categories = ["rail_freight", "rail_passenger"],
        magnitude_relative_to_baseline = True,
        model_energy = model_energy,
        strategy_id = strat
    )

    # add clean grid
    df_strat_cur = transformation_clean_grid(df_strat_cur, strat)

    df_all_transformations[ind_strategy] = df_strat_cur
    dict_strat_name_to_ind.update({strat_name: ind_strategy})
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




##  EN: All Transformations

strat_name = "EN: All Energy Transformations"
strat = dict_strat_to_strat_id.get(strat_name)

ind_strategy += 1
df_all_transformations.append(None)


if strat is not None:

    # add bundles
    df_strat_cur = all_transformations_fgtv(df_input, strat)
    df_strat_cur = all_transformations_inen(df_strat_cur, strat)
    df_strat_cur = all_transformations_trns(df_strat_cur, strat)
    df_strat_cur = all_transformations_scoe(df_strat_cur, strat)
    df_strat_cur = transformation_clean_grid(df_strat_cur, strat)

    # ENTC: Reduce transmission losses
    df_strat_cur = adt.transformation_entc_specify_transmission_losses(
        df_strat_cur,
        0.04,
        vec_implementation_ramp,
        sa.model_attributes,
        model_elec,
        model_energy = model_energy,
        strategy_id = strat
    )

    if False:
        # CCSQ: Increase direct air capture
        df_strat_cur = adt.transformation_ccsq_increase_direct_air_capture(
            df_strat_cur,
            50,
            vec_implementation_ramp,
            sa.model_attributes,
            model_energy = model_energy,
            strategy_id = strat
        )

    df_all_transformations[ind_strategy] = df_strat_cur
    print(f"Successfully added {attr_strat.key} = {strat}: {strat_name}")




################################
#    CONCATENATE AND EXPORT    #
################################

df_all_transformations = pd.concat(df_all_transformations, axis = 0)
if True:
    fp_out = dict_template_file_paths.get("en").replace(".csv", "_with_transformations.csv")
    print(f"writing to output at {fp_out}...")
    df_all_transformations.to_csv(
        fp_out,
        index = None,
        encoding = "UTF-8"
    )

print("Done.")



Successfully added strategy_id = 3001: FGTV: Minimize leaks
Successfully added strategy_id = 3002: FGTV: Maximize flaring
Successfully added strategy_id = 3003: FGTV: All Fugitive Emissions Transformations
Successfully added strategy_id = 3004: INEN: Maximize industrial energy efficiency
Successfully added strategy_id = 3005: INEN: Maximize industrial production efficiency
Successfully added strategy_id = 3006: INEN: Fuel switch low-temp thermal processes to industrial heat pumps
Successfully added strategy_id = 3007: INEN: Fuel switch low-temp thermal processes to industrial heat pumps with Renewable Grid
Successfully added strategy_id = 3008: INEN: Fuel switch medium and high-temp thermal processes to hydrogen and electricity
Successfully added strategy_id = 3009: INEN: Fuel switch medium and high-temp thermal processes to hydrogen and electricity with Renewable Grid and Green Hydrogen
Successfully added strategy_id = 3010: INEN: All Industrial Energy Transformations
Successfully add

In [48]:
df_check = df_all_transformations[
    df_all_transformations[field_iso].isin(["BRA"]) & 
    df_all_transformations["strategy_id"].isin([3020]) & 
    df_all_transformations["time_period"].isin(range(19,24))
]
#df_check[[x for x in df_check.columns if ("prodinit" in x) and ("electronics" in x)]]
l = [x for x in df_check.columns if ("gdp" in x)]
df_check[l]

,ef_ippu_tonne_c2f6_per_mmm_gdp_product_use_ods_other,ef_ippu_tonne_c2h3f3_per_mmm_gdp_product_use_ods_refrigeration,ef_ippu_tonne_c2hf5_per_mmm_gdp_product_use_ods_other,ef_ippu_tonne_c2hf5_per_mmm_gdp_product_use_ods_refrigeration,ef_ippu_tonne_c3h2f6_per_mmm_gdp_product_use_ods_other,ef_ippu_tonne_c3h2f6_per_mmm_gdp_product_use_ods_refrigeration,ef_ippu_tonne_c3h3f5_per_mmm_gdp_product_use_ods_other,ef_ippu_tonne_c3h3f5_per_mmm_gdp_product_use_ods_refrigeration,ef_ippu_tonne_c3hf7_per_mmm_gdp_product_use_ods_other,ef_ippu_tonne_c3hf7_per_mmm_gdp_product_use_ods_refrigeration,...,physparam_wali_tonne_cod_per_mmm_gdp,qty_wali_m3_ww_industrial_per_gdp,qty_waso_industrial_waste_kt_per_mmm_gdp,consumpinit_inen_energy_tj_per_mmm_gdp_other_product_manufacturing,gdp_mmm_usd,consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_elec_appliances,consumpinit_scoe_tj_per_mmmgdp_commercial_municipal_heat_energy,consumpinit_scoe_tj_per_mmmgdp_other_se_elec_appliances,consumpinit_scoe_tj_per_mmmgdp_other_se_heat_energy,elasticity_ippu_mining_production_to_gdp
55,1,1,1,1,1,1,1,1,1,1,...,1000,1000000.0,30,768.491160,3181.172701,159.611551,7.299729,35.243376,3.616614,0.5
56,1,1,1,1,1,1,1,1,1,1,...,1000,1000000.0,30,752.951268,3246.827790,156.383998,7.152119,34.530709,3.543482,0.5
57,1,1,1,1,1,1,1,1,1,1,...,1000,1000000.0,30,737.582350,3314.481563,153.191955,7.006133,33.825883,3.471154,0.5
58,1,1,1,1,1,1,1,1,1,1,...,1000,1000000.0,30,722.828289,3382.135336,150.127614,6.865987,33.149254,3.401719,0.5
59,1,1,1,1,1,1,1,1,1,1,...,1000,1000000.0,30,708.652912,3449.789110,147.183463,6.731338,32.499165,3.335008,0.5


In [ ]:
attr_ind = sa.model_attributes.dict_attributes.get("cat_industry")



In [44]:
df_check = df_all_transformations[
    df_all_transformations[field_iso].isin(["BRA"]) & 
    df_all_transformations["strategy_id"].isin([3008])
]

for cat in attr_ind.key_values:
    
    flds = [x for x in df_check.columns if x.startswith(f"frac_inen_energy_{cat}")]
    
    if len(flds) > 0:
        vec = np.array(df_check[flds].sum(axis = 1))
        tup = np.round(min(vec), decimals = 6), np.round(max(vec), decimals = 6)
        
        print(f"{cat}:\t{tup}")
    



agriculture_and_livestock:	(1.0, 1.0)
cement:	(1.0, 1.0)
chemicals:	(1.0, 1.0)
electronics:	(1.0, 1.0)
glass:	(1.0, 1.0)
lime_and_carbonite:	(1.0, 1.0)
metals:	(1.0, 1.0)
mining:	(1.0, 1.0)
other_product_manufacturing:	(1.0, 1.0)
paper:	(1.0, 1.0)
plastic:	(1.0, 1.0)
recycled_glass:	(1.0, 1.0)
recycled_metals:	(1.0, 1.0)
recycled_paper:	(1.0, 1.0)
recycled_plastic:	(1.0, 1.0)
recycled_rubber_and_leather:	(1.0, 1.0)
recycled_textiles:	(1.0, 1.0)
recycled_wood:	(1.0, 1.0)
rubber_and_leather:	(1.0, 1.0)
textiles:	(1.0, 1.0)
wood:	(1.0, 1.0)
